In [ ]:
# データ収集用Jupyterノートブック

import sys
import os
import json
import datetime as dt
import pandas as pd
import asyncio

# データコレクターモジュールをインポート
from data_collector import BTCDataCollector

# 設定ファイルから読み込む
def load_config(config_path="config/data_config.json"):
    try:
        with open(config_path, "r") as f:
            config = json.load(f)

        # 日付文字列をdatetimeオブジェクトに変換
        if "start_date" in config and config["start_date"]:
            config["start_date"] = dt.datetime.fromisoformat(config["start_date"])
        if "end_date" in config and config["end_date"]:
            config["end_date"] = dt.datetime.fromisoformat(config["end_date"])
        else:
            config["end_date"] = dt.datetime.now()

        return config
    except Exception as e:
        print(f"設定ファイルの読み込みエラー: {e}")
        return None

# 設定ファイルから読み込み
config = load_config()

# または直接設定
if config is None:
    config = {
        "exchange": "bybit",
        "symbol": "BTCUSDT",
        "timeframe": "5",
        "start_date": dt.datetime(2023, 1, 1),
        "end_date": dt.datetime.now(),
        "output_dir": "data/raw",
        "output_filename": "btcusd_5m_data.csv",
        "api_keys": {
            "bybit": ["YOUR_API_KEY", "YOUR_API_SECRET"]  # 実際のキーに置き換え
        },
        "use_direct_api": True  # True: 直接APIを使用, False: pybottersを使用
    }

# API キーを設定（必要な場合）
# config["api_keys"]["bybit"] = ["YOUR_ACTUAL_API_KEY", "YOUR_ACTUAL_API_SECRET"]

# 設定内容を表示（APIキーは隠す）
print_config = config.copy()
print_config["api_keys"] = {k: ["***", "***"] for k, v in config["api_keys"].items()}
print("使用する設定:")
for key, value in print_config.items():
    print(f"  {key}: {value}")

# データ収集と保存の関数
async def collect_and_save_data(config):
    """データを収集して保存する関数"""
    collector = BTCDataCollector(config)
    df = await collector.collect_historical_data()
    summary = collector.save_data(df)

    # サマリー出力
    print("\n=== データ収集結果サマリー ===")
    print(f"状態: {summary['status']}")
    print(f"行数: {summary['rows']}")
    print(f"期間: {summary['start_date']} から {summary['end_date']}")
    if summary['status'] == 'success':
        print(f"価格統計（終値）: 最小={summary['data_stats']['close']['min']:.2f}, "
              f"最大={summary['data_stats']['close']['max']:.2f}, "
              f"平均={summary['data_stats']['close']['mean']:.2f}")
        print(f"出来高合計: {summary['data_stats']['volume']['total']:.2f}")
        print(f"欠損値: {sum(summary['missing_values'].values())}")

    return df, summary

# データ収集実行
df, summary = await collect_and_save_data(config)

# データ確認
if 'df' in locals() and not df.empty:
    print("データサンプル（先頭5行）:")
    print(df.head())

    print("\n基本統計:")
    print(df.describe())

    print(f"\n合計レコード数: {len(df)}")
else:
    print("データの収集に失敗したか、データが空です。")

2025-04-26 01:15:55,438 - bybit_collector - INFO - BTCUSDTの5分足データを2023-01-01 00:00:00から2025-04-26 01:15:55.437296まで収集します
2025-04-26 01:15:55,438 - bybit_collector - INFO - データ取得を847個のチャンクに分割しました


使用する設定:
  exchange: bybit
  symbol: BTCUSDT
  timeframe: 5
  start_date: 2023-01-01 00:00:00
  end_date: 2025-04-26 01:15:55.437296
  output_dir: data/raw
  output_filename: btcusd_5m_data.csv
  api_keys: {'bybit': ['***', '***']}
  use_direct_api: True


データ取得:   0%|          | 0/847 [00:00<?, ?it/s]

2025-04-26 01:16:11,495 - bybit_collector - INFO - データ収集完了。合計 169215 行のデータを取得
2025-04-26 01:16:12,122 - bybit_collector - INFO - データを data\raw\btcusd_5m_data.csv に保存しました。データサイズ: (169215, 6)



=== データ収集結果サマリー ===
状態: success
行数: 169215
期間: 2022-12-31T22:25:00 から 2025-04-25T16:15:00
価格統計（終値）: 最小=16508.00, 最大=109059.40, 平均=53358.14
出来高合計: 65739578.68
欠損値: 0
データサンプル（先頭5行）:
                        open     high      low    close   volume      turnover
timestamp                                                                     
2022-12-31 22:25:00  16545.0  16568.5  16544.5  16557.0  199.789  3.308472e+06
2022-12-31 22:30:00  16557.0  16569.0  16557.0  16563.0  265.757  4.402056e+06
2022-12-31 22:35:00  16563.0  16563.0  16544.5  16546.0  165.466  2.738463e+06
2022-12-31 22:40:00  16546.0  16552.0  16528.5  16551.5  295.414  4.885751e+06
2022-12-31 22:45:00  16551.5  16552.0  16533.0  16534.0   78.281  1.295245e+06

基本統計:
                open           high            low          close  \
count  169215.000000  169215.000000  169215.000000  169215.000000   
mean    53357.860402   53405.965116   53309.085535   53358.138956   
std     25347.134130   25375.056161   25319.137308  